In [ ]:
import json
import re
import pandas as pd
from groq import Groq

with open(r"D:\CTS Claims project\fraud_explanation.json", "r") as f:
    explainable_output = json.load(f)

claim_id = explainable_output.get("claim_id", "N/A")
provider_id = explainable_output.get("provider_id", "N/A")
fraud_score = explainable_output["fraud_score"]
predicted_label = explainable_output["predicted_label"]
feature_contributions = explainable_output.get("feature_contributions", {})
metrics = explainable_output.get("metrics", {})

tp = metrics.get("tp", 0)
fp = metrics.get("fp", 0)
fn = metrics.get("fn", 0)
tn = metrics.get("tn", 0)
avg_claim_value = metrics.get("avg_claim_value", 0)
personnel_cost = metrics.get("personnel_cost", 0)
infra_cost = metrics.get("infra_cost", 0)
compliance_cost = metrics.get("compliance_cost", 0)

client = Groq(api_key="Your_API_KEY") 
def explain_claim_with_llm():
    # <<< ADDED: compute weightage for features >>>
    if feature_contributions:
        total_contribution = sum(abs(w) for w in feature_contributions.values()) or 1
        weighted_contributions = {
            feat: {
                "weight": weight,
                "percentage": (abs(weight) / total_contribution) * 100
            }
            for feat, weight in feature_contributions.items()
        }
        contrib_text = "\n".join([
            f"- {feat}: {vals['weight']:.4f} ({vals['percentage']:.2f}%)"
            for feat, vals in weighted_contributions.items()
        ])
    else:
        weighted_contributions = {}
        contrib_text = "No feature-level explanation available."
    # <<< END ADDED >>>

    explanation_prompt = f"""
    You are an expert fraud investigator. Explain this prediction in plain English for a business user.

    Claim Details:
    - Claim ID: {claim_id}
    - Provider ID: {provider_id}
    - Fraud Score: {fraud_score:.4f}
    - Prediction: {predicted_label}

    Why:
    {contrib_text}

    Metrics:
    - TP: {tp}, FP: {fp}, FN: {fn}, TN: {tn}
    - Avg Claim Value: ${avg_claim_value:,.2f}
    - Costs: Personnel ${personnel_cost}, Infra ${infra_cost}, Compliance ${compliance_cost}

    Task:
    Explain in 3–4 sentences why this claim was classified this way, using very simple language.
    """

    # <<< ADDED: send weightage also in table prompt >>>
    table_prompt = f"""
    Return ONLY a JSON array (no text, no markdown) with the following fields:
    - Claim ID
    - Provider ID
    - Fraud Score
    - Prediction
    - TP, FP, FN, TN these values should be not in the confusion matrix values, it should be a real time value (eg: 55 )
    - Avg Claim Value, this value should not be in minus, this should be the average of total amount claimed
    - Personnel Cost
    - Infra Cost
    - Compliance Cost
    - Top Feature Contributions (list of feature, weight, percentage)

    Use this data:
    Claim ID: {claim_id}
    Provider ID: {provider_id}
    Fraud Score: {fraud_score:.4f}
    Prediction: {predicted_label}
    TP: {tp}, FP: {fp}, FN: {fn}, TN: {tn}
    Avg Claim Value: {avg_claim_value}
    Personnel Cost: {personnel_cost}
    Infra Cost: {infra_cost}
    Compliance Cost: {compliance_cost}
    Top Feature Contributions: {json.dumps(weighted_contributions, indent=2)}
    """

    # <<< END ADDED >>>

    explanation_response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": explanation_prompt}],
        temperature=0
    )
    explanation = explanation_response.choices[0].message.content.strip()
    table_response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": table_prompt}],
        temperature=0
    )
    table_json = table_response.choices[0].message.content.strip()

    cleaned_json = re.sub(r"```json|```", "", table_json).strip()
    json_match = re.search(r"(\[.*\]|\{.*\})", cleaned_json, re.DOTALL)
    if json_match:
        cleaned_json = json_match.group(1)

    try:
        metrics_data = json.loads(cleaned_json)
        if isinstance(metrics_data, dict):
            metrics_data = [metrics_data]
    except json.JSONDecodeError:
        raise ValueError(f"Failed to parse JSON from LLM. Cleaned text:\n{cleaned_json}")
    df = pd.DataFrame(metrics_data)
    csv_file = r"D:\CTS Claims project\metrics_table.csv"
    json_file = r"D:\CTS Claims project\metrics_table.json"

    df.to_csv(csv_file, index=False)
    df.to_json(json_file, orient="records", indent=4)

    print(f"✅ Metrics saved as:\n- {csv_file}\n- {json_file}")

    return explanation, metrics_data

explanation, metrics_data = explain_claim_with_llm()

print("\n=== HUMAN-FRIENDLY EXPLANATION ===")
print(explanation)
print("\n=== METRICS JSON (from LLM) ===")
print(json.dumps(metrics_data, indent=4))


✅ Metrics saved as:
- D:\CTS Claims project\metrics_table.csv
- D:\CTS Claims project\metrics_table.json

=== HUMAN-FRIENDLY EXPLANATION ===
This claim was classified as "not fraudulent" because several factors suggest it's a normal claim. The claim amount is relatively low ($1000 or less) and the patient's length of stay in the hospital is short (5 days or less), which are both common characteristics of legitimate claims. Additionally, the procedure and diagnosis codes used are consistent with standard practices, further supporting the claim's legitimacy. Overall, the combination of these factors led to a low fraud score, indicating that this claim is likely genuine.

=== METRICS JSON (from LLM) ===
[
    {
        "Claim ID": "CLM0000000001",
        "Provider ID": "PRV001253",
        "Fraud Score": 0.1915,
        "Prediction": "NON-FRAUD",
        "TP": 55,
        "FP": 65,
        "FN": 75,
        "TN": 85,
        "Avg Claim Value": 1234.56,
        "Personnel Cost": 5000,
   